In [55]:
# !pip install -r "./src/requirements.txt"

In [56]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
import tabulate


load_dotenv()



True

In [3]:
# os.getenv("OPENAI_API_KEY")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 424, 'total_tokens': 433}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-b48e3942-e9ea-42a7-a31e-dac7ef7bc32d-0', usage_metadata={'input_tokens': 424, 'output_tokens': 9, 'total_tokens': 433})

In [17]:
parser = StrOutputParser()
parser.invoke(model.invoke(messages))

'Hello! How can I assist you today?'

In [73]:
def load_dataframes(file_paths):
  dataframes = {}
  for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes[file_name] = df

  return dataframes

path=["./src/data/insurance_data.csv"]
# path=["hi", "How"]
dataframes= load_dataframes(path)

# Passing csv info as markdown 
schema_and_rows = []
for name, df in dataframes.items():
    schema = df.dtypes.to_dict()
    top_3_rows = df.head(3).to_markdown(index=False)
    schema_md = "\n".join([f"- **{col}**: {dtype}" for col, dtype in schema.items()])
    schema_and_rows.append(f"### DataFrame Name: {name}\n\n**Schema:**\n{schema_md}\n\n**Top 3 Rows:**\n{top_3_rows}\n")

schema_and_rows_str = "\n".join(schema_and_rows)



model= ChatOpenAI(model="gpt-4o-mini")

def genrate_system_prompt(schema_and_rows_str):
    system_message=f"""
    You are an intelligent assistant designed to help users analyze data using a provided dataframe. Your primary tasks are:
    **Genral Answering**: If a user's question can be directly answered without using the dataframes information.
    1. **Direct Answering**: If a user's question can be directly answered using the dataframe, identify the relevant columns and provide the answer.
    2. **Query Rewriting**: If the question cannot be directly answered, rewrite the query to include the correct column names from the dataframe, ensuring that the query is clear and can be used to retrieve the necessary information.
    
    Here are the dataframes for your reference:
    {schema_and_rows_str}

    Examples:
    1. **Direct Answering**:
    - User Question: "What is the average sales for the last quarter?"
    - DataFrame Schema: { 'Date', 'Sales', 'Product', 'Region' }
    - Thought: "From sales column in the dataframe I can find the average sales for the last quarter."

    2. **Query Rewriting**:
    - User Question: "What is the performance of the product?"
    - DataFrame Schema: { 'Date', 'Sales', 'Product', 'Region' }
    - Thought: "To evaluate the performance of a product, I can analyze the 'Sales' column filtered by the 'Product' column. For example, 'What are the total sales for each product over the last year?

      When responding to user queries, follow these steps:
      1. **Understand the Question**: Parse the user's question to understand what information they are seeking.
      2. **Identify Relevant Columns**: Determine which columns in the dataframes are relevant to the question.
      3. **Provide Structured Response**: Based on the question type, provide a response in the following structure:
        - response_type: 'general', 'direct', or 'rewriting'
        - answer: The answer to the question or the rewritten query
        - relevant_columns: List of column names used in the answer (if applicable)
        - confidence: Your confidence in the answer (0-1)

    Always ensure that your responses are clear, concise, and actionable."""
    
    # system_message=f"""
    # You are an intelligent assistant designed to help users analyze data using a provided dataframe. Your primary tasks are:
    # **Genral Answering**: If a user's question can be directly answered without using the dataframe
    # 1. **Direct Answering**: If a user's question can be directly answered using the dataframe, identify the relevant columns and provide the answer.
    # 2. **Query Rewriting**: If the question cannot be directly answered, rewrite the query to include the correct column names from the dataframe, ensuring that the query is clear and can be used to retrieve the necessary information.
    
    # Here are the dataframes for your reference:
    # {schema_and_rows_str}

    # When responding to user queries, follow these steps:
    # 1. **Understand the Question**: Parse the user's question to understand what information they are seeking.
    # 2. **Identify Relevant Columns**: Determine which columns in the dataframe are relevant to the question.
    # 3. **Direct Answering**: If possible, use the dataframe to directly answer the question.
    # 4. **Query Rewriting**: If a direct answer is not possible, rewrite the user's query to include the correct column names and provide a clear, actionable query.

    # Examples:
    # 1. **Direct Answering**:
    # - User Question: "What is the average sales for the last quarter?"
    # - DataFrame Schema: { 'Date', 'Sales', 'Product', 'Region' }
    # - Direct Answer: "The average sales for the last quarter is $X."

    # 2. **Query Rewriting**:
    # - User Question: "What is the performance of the product?"
    # - DataFrame Schema: { 'Date', 'Sales', 'Product', 'Region' }
    # - Query Rewriting: "To evaluate the performance of a product, you can analyze the 'Sales' column filtered by the 'Product' column. For example, 'What are the total sales for each product over the last year?"""
    
    return system_message

system_message= SystemMessage(content=genrate_system_prompt(schema_and_rows_str))

human_message= HumanMessage(content="What is the average premium across all policies?")
messages=[(system_message), (human_message)]

model.invoke(messages)



AIMessage(content="To find the average premium across all policies, we can look at the 'Premium' column in the dataframe.\n\nThe average premium can be calculated as follows:\n- Average Premium = Sum of all Premiums / Number of Policies\n\nLet me calculate that for you.\n\nThe average premium across all policies is $1,736.35.", response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 798, 'total_tokens': 864}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb343f3b-3ec3-482a-b354-9b2e93afae60-0', usage_metadata={'input_tokens': 798, 'output_tokens': 66, 'total_tokens': 864})

In [76]:
# getting structured output

from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class QueryType(BaseModel):
    """Return the Querty type either of one result: [General, Direct, Rewriting]"""

    query_type: str = Field(description="Return  either General, Direct or Rewriting")

structured_llm = model.with_structured_output(QueryType)

structured_llm.invoke("What is the average premium across all policies?")

QueryType(query_type='average premium across all policies')